In [150]:
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.stats import norm
import tqdm

In [12]:
#Q3
T=1
N=252
K=110
r=0.05
h=T/N

# Create Function:
def CI(data,alpha):
    sample_mean=np.mean(data) # data is a list!
    sample_sigma=np.std(data)
    critical_point = norm.ppf(1-alpha/2)
    LB=sample_mean-critical_point*sample_sigma/np.sqrt(len(data))
    UB=sample_mean+critical_point*sample_sigma/np.sqrt(len(data))
    return LB,UB

def Q3_simulation(size,significant_level):
    call=[]
    for i in tqdm.tqdm(range(0,size)):
        St=100
        for j in range(0,N):
            Z=np.random.normal(0,1)
            sigma = 0.5*np.exp(-j*h)*((100/St)**0.3)
            St=St+r*St*h+sigma*St*np.sqrt(h)*Z
        call.append(np.max([St-K,0])*np.exp(-r*T)) #European option 所以只需要最後一期
    return np.mean(call),CI(call,significant_level)

In [15]:
Q3_simulation(1000000,0.05)

100%|██████████| 1000000/1000000 [20:27<00:00, 814.71it/s]


(11.03031291905867, (10.990655325916336, 11.069970512201003))

In [75]:
#Q5
theta=-0.1
sigma=0.2
v=0.2
S0=[100]
r=0.03
K=100
T1=0.25
T2=0.5
N=252
h=T2/N
delta=0
w=(1/v)*np.log(1-theta*v-(sigma**2)*v/2)

# set simulation
def Q5_simulation(size,significant_level):
    payoff=[]
    for i in tqdm.tqdm(range(0,size)):
        Z=np.random.normal(0,1,N)
        g=np.random.gamma(h/v,v,N)
        # generate X (list)
        X=theta*g+sigma*np.sqrt(g)*Z
        # generate S
        St=S0[0]*np.exp(np.cumsum((r-delta+w)*h+X))
        St=S0+list(St) #併入S0
        St1=St[int(T1/h)] # 找到T1的Stock price
        b = np.max([St[-1]-np.min([K,St1]),0])*np.exp(-r*T2)
        payoff.append(b)
    # let payoff discounted from T2 to T=0 to get our price of contract.
    return np.mean(payoff),CI(payoff,significant_level)

In [79]:
Q5_simulation(5000000,0.05)

100%|██████████| 5000000/5000000 [06:08<00:00, 13572.07it/s]


(7.101765832561304, (7.093754765735225, 7.109776899387382))

In [9]:
#Q4
r=0.03
k=2
theta=0.04
sigma=0.5
rho=-0.7
K=120
T=0.5
N=252
h=T/N

def Q4_simulation(m,size,significant_level):
    price=[]
    for j in range(0,size):
        S=[100]
        Stm=[]
        v=[0.04]
        # generate independent Z
        Z1=np.random.normal(0,1,N)
        Z2=np.random.normal(0,1,N)
        # By formula, Zv=Z1, Zs=rho*Z1+np.sqrt(1-(rho**2))*Z2
        Zv=Z1
        Zs=rho*Z1+np.sqrt(1-(rho**2))*Z2
        # Generate v
        for i in range(0,N):
            a=v[i]+k*(theta-np.max([v[i],0]))*h+sigma*np.sqrt(np.max([v[i],0]))*np.sqrt(h)*Zv[i]
            v.append(a)
        # Generate S
            b=S[i]*np.exp((r-(v[i]/2))*h+np.sqrt(np.max([v[i],0])*h)*Zs[i])
            S.append(b)
        for i in range(1,m+1):
            c=int(i*N/m)
            Stm.append(S[c])
        price.append(np.max([np.max(Stm)-K,0])*np.exp(-r*T))
    return np.mean(price),CI(price,significant_level)

In [10]:
Q4_simulation(3,1000,0.05)

(0.25626933519454487, (0.1719137961584551, 0.34062487423063464))

In [11]:
Q4_simulation(6,1000,0.05)

(0.27722796017924173, (0.18048996193323977, 0.3739659584252437))

In [12]:
Q4_simulation(12,1000,0.05)

(0.3281861920238675, (0.22174885186287305, 0.43462353218486194))

In [151]:
def Q4_simulation_2(m,size,significant_level):
    r=0.03
    k=2
    theta=0.04
    sigma=0.5
    rho=-0.7
    K=120
    T=0.5
    N=252
    h=T/N
    v0 = 0.04
    S0 = 100
    # 矩陣作法代替Monte Carlo最外層的迴圏
    v = np.ones((1,size))*v0
    zero = np.zeros(size)
    S = np.ones((1,size))*S0 
    # generate independent Z
    Z1=np.random.normal(0,1,(N,size))
    Z2=np.random.normal(0,1,(N,size))
    # By formula, Zv=Z1, Zs=rho*Z1+np.sqrt(1-(rho**2))*Z2
    Zv=Z1
    Zs=rho*Z1+np.sqrt(1-(rho**2))*Z2
    for i in tqdm.tqdm(range(N)):
        a=v[i,:]+k*(theta-np.max([v[i,:],zero],axis=0))*h+sigma*np.sqrt(np.max([v[i,:],zero],axis=0))*np.sqrt(h)*Zv[i,:]
        v = np.append(v,[a],axis=0)
        b=S[i,:]*np.exp((r-(v[i,:]/2))*h+np.sqrt(np.max([v[i,:],zero],axis=0)*h)*Zs[i,:])
        S = np.append(S,[b],axis=0)
    for i in range(1,m+1):
        c=int(i*N/m)
        if i==1:
            Stm = [S[c,:]]
        else:
            Stm = np.append(Stm,[S[c,:]],axis=0)
    price = np.max([np.max(Stm,axis=0)-K,zero],axis=0)*np.exp(-r*T)
    return np.mean(price),CI(price,significant_level)

In [154]:
Q4_simulation_2(3,1000000,0.05)

100%|██████████| 252/252 [04:38<00:00,  1.11s/it]


(0.25633599724903766, (0.25337882229994413, 0.2592931721981312))

In [155]:
Q4_simulation_2(6,500000,0.05)

100%|██████████| 252/252 [02:13<00:00,  1.89it/s]


(0.2756311108200495, (0.27133518142205787, 0.2799270402180411))

In [157]:
Q4_simulation_2(12,700000,0.05)

100%|██████████| 252/252 [03:05<00:00,  1.36it/s]


(0.30052468525477616, (0.2966885542099131, 0.30436081629963924))

In [13]:
# Q2 
# Parameters
r=0.04
sigma=0.3
K=55
T=1
i0=2
m=4 ## Notice!
N=252

def Stockprice(N):
    S=[100]
    Z=np.random.normal(0,1,N)
    h=T/N
    for j in range(0,N):
        a=S[j]+r*S[j]*h+sigma*S[j]*np.sqrt(h)*Z[j]
        S.append(a)
    return S

def coupon(c,tao_period): # c : Annual coupon(單位:%)
    coupon = 0
    for i in range(1,tao_period+1):
        coupon=coupon+np.exp(-i*r*T/m)*(c*T/m)
    return coupon

def Q2_simulation(c,size,significant_level):
    VC=[]
    for j in range(0,size):
        S = Stockprice(N) # use function of stock price simulation
        # 
        period=[]
        for i in range(i0,m+1):
            if S[int(i*N/m)]>=S[0]:
                period.append(i)
            else:
                period.append(m)
        tao_period = min(period) # 注意tao是期數

        # payoff # use coupon function
        if tao_period<m:
            payoff = np.exp(-r*T*tao_period/m)*S[0]+coupon(c,tao_period)
        else:
            if S[-1]>K:
                payoff = np.exp(-r*T)*S[0]+coupon(c,tao_period)
            else:
                payoff = np.exp(-r*T)*S[-1]+coupon(c,tao_period)
        VC.append(payoff)
    return CI(VC,significant_level)

In [14]:
Q2_simulation(5.66,1000000,0.05)

(99.97309178124375, 100.00236261193247)